<a href="https://colab.research.google.com/github/eetoorivarshika06/Aushad/blob/main/Copy_of_PD_Expected_Loss_Model_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
gas_df = pd.read_csv("Nat_Gas.csv")
gas_df.head()


,Dates,Prices
0,10/31/20,10.1
1,11/30/20,10.3
2,12/31/20,11.0
3,1/31/21,10.9
4,2/28/21,10.9


In [ ]:
gas_df["Dates"] = pd.to_datetime(gas_df["Dates"])

price_dict = {
    row["Dates"].strftime("%Y-%m-%d"): row["Prices"]
    for _, row in gas_df.iterrows()
}


/tmp/ipython-input-2079725687.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  gas_df["Dates"] = pd.to_datetime(gas_df["Dates"])


In [ ]:
def price_storage_contract(
    injection_dates,
    withdrawal_dates,
    prices,
    injection_rate,
    withdrawal_rate,
    max_storage,
    storage_cost_per_day
):
    events = []

    for d in injection_dates:
        events.append((pd.to_datetime(d), "inject"))

    for d in withdrawal_dates:
        events.append((pd.to_datetime(d), "withdraw"))

    events.sort(key=lambda x: x[0])

    stored_volume = 0
    contract_value = 0
    last_date = events[0][0]

    for date, action in events:
        days_passed = (date - last_date).days
        contract_value -= stored_volume * storage_cost_per_day * days_passed

        price = prices[date.strftime("%Y-%m-%d")]

        if action == "inject":
            volume = min(injection_rate, max_storage - stored_volume)
            stored_volume += volume
            contract_value -= volume * price

        else:
            volume = min(withdrawal_rate, stored_volume)
            stored_volume -= volume
            contract_value += volume * price

        last_date = date

    return contract_value


In [ ]:
injection_dates = ["2021-01-31", "2021-02-28"]
withdrawal_dates = ["2021-06-30", "2021-07-31"]

injection_rate = 1000000
withdrawal_rate = 1000000
max_storage = 2000000
storage_cost_per_day = 0.01


In [ ]:
contract_value = price_storage_contract(
    injection_dates,
    withdrawal_dates,
    price_dict,
    injection_rate,
    withdrawal_rate,
    max_storage,
    storage_cost_per_day
)

print("Storage Contract Value:", contract_value)


Storage Contract Value: -4730000.0


A prototype pricing model was built to value a natural gas storage contract by explicitly modeling all cash flows. Injection dates incur purchase costs while withdrawal dates generate sales revenue. Storage costs accrue based on stored volume and time. Capacity and injection/withdrawal rate constraints are enforced. With zero interest rates assumed, the contract value is calculated as the sum of all undiscounted cash flows.